In [1]:
import unicodedata

from bs4 import BeautifulSoup
import sqlite3
import requests
import re
import json
import pandas as pd

class scrapper:
    @staticmethod
    def get_arks(year):
        url_base = "https://gallica.bnf.fr/ark:/12148/cb34363188x/date"+str(year)+"0101"
        req = requests.get(url_base, headers={'User-Agent': 'Mozilla/5.0'})
        p = req.content
        L = re.findall(r"https://gallica.bnf.fr/ark:/(\d*/(bpt\w*))?",str(p))
        S = set()
        for i in L:
            if(i[1] != ""):
                S.add(i[1])
        return S
    @staticmethod
    def get_pagination(ark):
        """fonction qui renvoie le nombre de pages d'un documents"""
        #print(cpt,"--> https://gallica.bnf.fr/"+i+"/f7/highres")
        req = requests.get("https://gallica.bnf.fr/services/Pagination?ark=" + ark)
        p = req.content
        pages = re.search(r"<nbVueImages>(\d*)</nbVueImages>", str(p))
        return pages.groups()[0]
    @staticmethod
    def get_page(ark, page, number = 1):
        
        req = requests.get(f"https://gallica.bnf.fr/ark:/12148/{ark}/f{page}n{number}.texteBrut")
        return req.content
    @staticmethod
    def get_document(ark, mode = "texteBrut"):
        req = requests.get("https://gallica.bnf.fr/ark:/12148/"+ark+"."+ mode)
        return req.content
    

In [119]:
class Cleaner:
    def __init__(self, directory):
        self.directory = directory
        pass
    def extract(self, file):
        soup = BeautifulSoup(file , "html.parser")
        df = pd.DataFrame(columns=["page", "arrêt", "date", "juridiction"])
        Decision, notes, page, new_page, new_decision, count = False, False, 0, True, False,1
        for tag in soup.body :
            if count == 15:
                count = 0
                Decision = False
            new_decision = False
            string = tag.get_text()
            if tag.name == "hr":
                page += 1
                notes = False
                new_page  = True
            if tag.name == "p" and string is not None and not new_page:
                m1 = re.match(r"(^.*?(La Cour,|L(A|À|a) COUR)(?! DE)(.+)$|^JUGEMENT\.?\s?$|^A\s?R\s?R\s?(Ê|E)\s?T\.\s?$)", string)
                m2 = re.match(r"(.*?)D(u|û|ù)(.+?)(—|–|-|–|–)(.+)", string ) 
                
                if not Decision and m1:
                    Decision = True
                    text = ""
                    count = 1
                    if m1.groups()[3] != None:
                        text = str(m1.groups()[3])
                    First_page = page
                    new_decision = True
                if Decision and m2:
                    if count < 15:
                        if(new_decision):
                            text =m2.groups()[0]
                        else:
                            text += m2.groups()[0]
                        date = m2.groups()[2]
                        juridiction = m2.groups()[4]
                        df = df.append({'page' : First_page, 'arrêt' : text , "date": date, "juridiction" : juridiction},ignore_index=True)
                    Decision = False
                    text = ''
                elif not notes and Decision and not new_decision:
                    if not re.match(r"^\(\d*\).+$", string):
                        count +=1
                        text+= string + "\n"
                    else:
                        notes = True
                else:
                    pass
            else :
                new_page = False
        return df
    def save(self, df, ark):
        df.to_csv(f"{self.directory}/{ark}.csv", encoding="utf-8")
        pass
    def postProcess(self, df):
        # fix mix date-juridiction
        Rows_contains_ = df['date'].str.contains(r"(—|–|-)")
        for i, row in df[Rows_contains_].iterrows():
            m = re.search(r"(.+?)(—|–|-|—)(.+)(—|–|-|—)?.*", row["date"] )
            if m:
                df.at[i, "date"] = m.groups()[0]
                df.at[i, "juridiction"] = m.groups()[2]
        #if still not fixed --> drop them
        Rows_contains_ = df['date'].str.contains(r"(—|–|-)")
        df = df[Rows_contains_ == False]
        # drop date too long
        leng = df["date"].str.len()
        df = df[leng < 25] # drop too long date
        # drop date with no number
        number = df["date"].str.contains("^\D*$")
        df = df[number== False] # drop too long date
        length_decision = df.arrêt.str.len()
        # drop decision too short
        df = df[length_decision > 100]
        # drop juridiction too long
        for i , row in df.iterrows():
            m = re.search(r"(.+?)(—|–|-|—|,|;).*", row["juridiction"] )
            if m:
                df.at[i, "juridiction"] = m.groups()[0]
        
        # process the juridiction
        
        return df              
        
                
  

In [126]:
years = scrapper.get_arks(1817)
years

{'bpt6k5791611b', 'bpt6k5791626s'}

In [131]:
test = scrapper.get_page("bpt6k5791626s", 1, 50 )

In [56]:
c = Cleaner("test")
file = scrapper.get_document("bpt6k5548364r")


In [132]:
c = Cleaner("test")
df_test = c.extract(test)
df_test

,page,arrêt,date,juridiction
0,6,", -Faisant droit sur l'appel interjeté par Bi...",n mars 1816.,Cour royale de Paris. - Prés. M. Séguier - Co...
1,8,"LA COUR, - Considérant qu'au 15 ventôse an 6, ...",6,avril 1816.- Cour royale de Paris.
2,9,"LA COUR , - Statuant sur le renvoi ordonné par...",18 mars 1816.,Cour royale de Paris. - Plaid. MM. Gauthier e...
3,10,"LA COUR , - Considérant que d'après l'art. 559...",n décembre 1815.,Cour de Colmar.
4,14,"LA COUR, - Adoptant les motifs des premiers ju...",II janvier 1816.,Cour royale de Paris. - V. chambre.-Présid. M...
5,15,"» LA COUR, - Joint l'appel principal interjeté...",11 mai 1816.,Cour royale de Paris.
6,20,"LA COUR, - Attendu que le délai de trois jours...",21 mars 1815.,Cour royale d'Angers.
7,45,"LA COUR, - Attendu que la fin de non-recevoir ...",22 avril 1815.,Cour royale de Colmar.
8,47,"LA COUR, - Considérant que l'adjudication du d...",16 mars 1816,Cour royale de Paris. - 3e. chambre Prés. M. ...
9,48,"LA COUR, - Adoptant les motifs des premiers ju...",2 mai 1816.,Cour royale de Paris.


In [133]:
df_test = c.postProcess(df_test)

In [134]:
df_test

,page,arrêt,date,juridiction
0,6,", -Faisant droit sur l'appel interjeté par Bi...",n mars 1816.,Cour royale de Paris.
1,8,"LA COUR, - Considérant qu'au 15 ventôse an 6, ...",6,avril 1816.
2,9,"LA COUR , - Statuant sur le renvoi ordonné par...",18 mars 1816.,Cour royale de Paris.
3,10,"LA COUR , - Considérant que d'après l'art. 559...",n décembre 1815.,Cour de Colmar.
4,14,"LA COUR, - Adoptant les motifs des premiers ju...",II janvier 1816.,Cour royale de Paris.
5,15,"» LA COUR, - Joint l'appel principal interjeté...",11 mai 1816.,Cour royale de Paris.
6,20,"LA COUR, - Attendu que le délai de trois jours...",21 mars 1815.,Cour royale d'Angers.
7,45,"LA COUR, - Attendu que la fin de non-recevoir ...",22 avril 1815.,Cour royale de Colmar.
8,47,"LA COUR, - Considérant que l'adjudication du d...",16 mars 1816,Cour royale de Paris.
9,48,"LA COUR, - Adoptant les motifs des premiers ju...",2 mai 1816.,Cour royale de Paris.


In [94]:
def clean_date(row):
    print(row)
    m = re.search(r"^(.+?)—(\D+\.?)*—", row["date"] )
    if m.groups() != None:
        return m.groups()[0]